# Experimentando difententes métodos com OTSU para calcular o LBP

> a imagem é disposta como uma matriz no qual os pixels variam de 0 a 1.

- menor: pixels abaixo do limiar de OTSU se tornam 0;
- maior: pixels acima ou igual ao limiar de OTSU se tornam 1;
- ambos (_default_): pixels abaixo do limiar de OTSU se tornam 0, os demais se tornam 1;

# Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd

from PIL import Image
from zipfile import ZipFile
from skimage.io import imread
from skimage.filters import threshold_otsu
from skimage.feature import local_binary_pattern

from img_edit import preprocessing_img

# Parâmetros do LBP
METHOD = 'nri_uniform'
RADIUS = 1
N_POINTS = 8
ZIP_PATH = 'patologias_toras.zip'
TARGET_NAMES = {'normal': 0,
                'broca':1,
                'fissura': 2,
                'musgo': 3}

In [2]:
path_dict = {name: [] for name in TARGET_NAMES}

with ZipFile(ZIP_PATH) as zip_file:
  for path in zip_file.namelist():
    # a pasta de cada amostra é o segundo elemento do split
    folder = path.split('/')[1]
    # Separando amostras por classe
    if '__MAC' not in path \
        and '.JPG' in path \
        and folder in TARGET_NAMES:
      path_dict[folder].append(f'./{ZIP_PATH}/{path}')

# Mostrando quantidade de amostras por classe
qtd_amostras_p_classe = []
for target in path_dict:
  n_amostras = len(path_dict[target])
  print(f'{target}: {n_amostras} amostras')

  qtd_amostras_p_classe.append(n_amostras)

N_MIN_DE_AMOSTRAS = min(qtd_amostras_p_classe)

normal: 107 amostras
broca: 90 amostras
fissura: 109 amostras
musgo: 111 amostras


In [3]:
def menor(arr, limiar) -> np.ndarray:
  img_otsu = arr.copy()
  img_otsu[arr<limiar] = 0
  return img_otsu

def maior(arr, limiar) -> np.ndarray:
  img_otsu = arr.copy()
  img_otsu[arr>=limiar] = 1
  return img_otsu

def ambos(arr, limiar) -> np.ndarray:
  return arr >= limiar

func_list = [(ambos, []),
             (menor, []),
             (maior, [])]

In [4]:
target_list = []
for target_name in TARGET_NAMES:
  path_list = path_dict[target_name]
  path_list_slice = path_list[:N_MIN_DE_AMOSTRAS]

  count = 0
  for path in path_list_slice:
    img = imread(path)
    img_prepro = preprocessing_img(img)

    if len(img_prepro) == 0:
      print(f'Erro no crop com imagem: ({path})')
      continue
      # pode acontecer do crop horizontal recortar a imagem toda
      # acontece erro com otsu, pula então a iteração
      # N_MIN_DE_AMOSTRAS -= 1

    # Salvando imagens
    img_name = path.split('/')[-1].lower()
    Image.fromarray(img_prepro, mode='L').save(f'./imgs/prepro_2/{target_name}-{img_name}.jpeg')

    target_list.append(TARGET_NAMES[target_name])

    for func, amostras in func_list:
      img_otsu = func(img_prepro, threshold_otsu(img_prepro))

      lbp = local_binary_pattern(image=img_otsu, P=N_POINTS, R=RADIUS, method=METHOD).flatten()
      
      arr = pd.Series(lbp).value_counts().sort_index().values
      arr = arr / arr.max()

      amostras.append(arr)

for func, amostras in func_list:
  data = pd.DataFrame(amostras)
  data['target'] = target_list
  data.dropna()
  data.to_csv(f'./csvs/prepro_2_{func.__name__}.csv', index=False)

pd.Series(target_list).value_counts()

<string>:6: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.


Erro no crop com imagem: (./patologias_toras.zip/patologias_toras/fissura/toras325.JPG)
Erro no crop com imagem: (./patologias_toras.zip/patologias_toras/musgo/toras75.JPG)


1    90
0    90
3    89
2    89
dtype: int64